In [1]:
import cv2
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [ ]:
def getLandmarks(image):
    face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    image.flags.writeable = False
    results = face_mesh.process(image)
    landmarks = results.multi_face_landmarks[0].landmark
    face_mesh.close()
    return landmarks, results

In [ ]:
def getRightEyeRect(image, landmarks):
    eye_top = int(landmarks[257].y * image.shape[0])
    eye_left = int(landmarks[362].x * image.shape[1])
    eye_bottom = int(landmarks[374].y * image.shape[0])
    eye_right = int(landmarks[263].x * image.shape[1])

    x = eye_left
    y = eye_top
    return x-20, y-20, eye_right+20, eye_bottom+20

In [ ]:
def getLeftEyeRect(image, landmarks):
    eye_top = int(landmarks[159].y * image.shape[0])
    eye_left = int(landmarks[33].x * image.shape[1])
    eye_bottom = int(landmarks[145].y * image.shape[0])
    eye_right = int(landmarks[133].x * image.shape[1])

    x = eye_left
    y = eye_top
    return x-20, y-30, eye_right+20, eye_bottom+30

In [ ]:
import os
import numpy as np
import random
import cv2
import csv

random.seed(42)

DATASET_DIR = 'Columbia Gaze Data Set'
destination_dir = "Columbia DataSet/"

def get_label_from_filename(filename):
    parts = filename.split('_')
    parts[-1] = parts[-1].split('.')[0]
    pose = float(parts[-3][:-1])
    horizontal_angle = float(parts[-1][:-1])
    return np.array([horizontal_angle, pose])

def load_images_and_labels(dataset_dir):
    with open(os.path.join(destination_dir, '_annotations.csv'), mode="a", newline="") as file:
        writer = csv.writer(file)

        for subject_dir in os.listdir(dataset_dir):

                subject_path = os.path.join(dataset_dir, subject_dir)
                if os.path.isdir(subject_path):
                    for img_name in os.listdir(subject_path):
                        img_path = os.path.join(subject_path, img_name)
                        
                        image = cv2.imread(img_path)
                        if image is None:
                            continue
                        
                        label = get_label_from_filename(img_path)
                        
                        try:
                            landmark, results = getLandmarks(image)
                        except:
                            continue
                        
                        landmarks = results.multi_face_landmarks[0].landmark
                        
                        height, width, _ = image.shape
                        
                        x_min, y_min = width, height
                        x_max, y_max = 0, 0
                        
                        for landmark in landmarks:
                            x, y = int(landmark.x * width), int(landmark.y * height)
                            x_min, y_min = min(x_min, x), min(y_min, y)
                            x_max, y_max = max(x_max, x), max(y_max, y)
                        
                        xtx = image[y_min:y_max, x_min:x_max].copy()
                        
                        try:
                            landmark, results = getLandmarks(xtx)
                        except:
                            continue
                        
                        xe, ye, we, he = getLeftEyeRect(xtx, landmark)
                        xe2, ye2, we2, he2 = getRightEyeRect(xtx, landmark)
                        
                        data = [img_name, label[0], xe, ye, we, he, xe2, ye2, we2, he2]
                        
                        writer.writerow(data)
                        
                        cv2.imwrite(os.path.join(destination_dir, img_name), xtx)
                        
                        del image, xtx
                        cv2.destroyAllWindows()

load_images_and_labels(DATASET_DIR)


XXXXXXXX     0024


I0000 00:00:1730176919.831843  332353 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1730176919.833938  370382 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730176919.836345  370385 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1730176919.848233  332353 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1730176919.849205  370392 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730176919.851436  370392 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1730176919.888345  332353 gl_context